# Complete pre-processing

## Importing libraries

In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
from xlsxwriter import Workbook
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier


## Loading the Dataset

In [ ]:
# Load the dataset
file_path = 'Dataset.xlsx'
dataset = pd.read_excel(file_path)

## Data Preprocessing

### Add an Index Column

In [16]:
dataset.reset_index(inplace=True)
dataset.rename(columns={'index': 'Index'}, inplace=True)
dataset['Index'] = dataset['Index'] + 1

In [17]:
# Rename the 'Index' column to 'Subject_ID' 
dataset = dataset.rename(columns={'Index': 'Subject_ID'})

### Outcome

In [ ]:
# Count the number of missing values in the 'outcome' column
missing_outcome_count = dataset['outcome'].isnull().sum()

# Identify the rows with missing values in the 'outcome' column
missing_outcome_rows = dataset[dataset['outcome'].isnull()]

print("Number of missing values in 'outcome':", missing_outcome_count)
print("Rows with missing 'outcome' values:\n", missing_outcome_rows)


In [19]:
# Creating a dictionary for mapping the categories to numerical values
outcome_mapping = {
    'discharge': 1,
    'death': 2,
    'transfer': 3
}

# Mapping the categories to numbers
dataset['Outcome_numerical'] = dataset['outcome'].map(outcome_mapping)

# Display the first few rows to verify the transformation
dataset[['outcome', 'Outcome_numerical']].head()

# Dropping the original 'outcome' column
dataset = dataset.drop('outcome', axis=1)


In [20]:

dataset['Outcome_numerical'].value_counts()

1.0    429
2.0     55
3.0      1
Name: Outcome_numerical, dtype: int64

### Gender Column 

In [21]:
# Correcting the gender column
dataset['GENDER'] = dataset['GENDER'].replace({'Μ': 'M'})

# Map the 'GENDER' column to numerical values: 'F' to 1 and 'M' to 2
dataset['GENDER'] = dataset['GENDER'].map({'F': 1, 'M': 2})


### Age Column

In [22]:
# Function to calculate age from birth date
def calculate_age(born):
    today = datetime.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

# Apply the function to the 'AGE' column
dataset['AGE'] = dataset['AGE'].apply(lambda x: calculate_age(x) if isinstance(x, datetime) else x)
"""
# Check the number of missing values in the original 'AGE' column
missing_ages_original = dataset['AGE'].isnull().sum()

# Impute missing values in the 'AGE' column with the median age before categorization
median_age = dataset['AGE'].median()
dataset['AGE'] = dataset['AGE'].fillna(median_age)

# Verify if all missing values are filled
missing_ages_after_imputation = dataset['AGE'].isnull().sum()

(missing_ages_original, missing_ages_after_imputation, median_age)
"""

"\n# Check the number of missing values in the original 'AGE' column\nmissing_ages_original = dataset['AGE'].isnull().sum()\n\n# Impute missing values in the 'AGE' column with the median age before categorization\nmedian_age = dataset['AGE'].median()\ndataset['AGE'] = dataset['AGE'].fillna(median_age)\n\n# Verify if all missing values are filled\nmissing_ages_after_imputation = dataset['AGE'].isnull().sum()\n\n(missing_ages_original, missing_ages_after_imputation, median_age)\n"

### LOS column

In [23]:
# FIxing some values
dataset['LOS'] = dataset['LOS'].replace('26+', 26)


### 'DAYS OF SYMPTOMS' Column

In [24]:
# Replace 'ΝΟ' with NaN in the 'DAYS OF SYMPTOMS' column
dataset['DAYS OF SYMPTOMS'] = dataset['DAYS OF SYMPTOMS'].replace('ΝΟ', 0)

### INFILTRATE COLUMN

In [25]:
# Redefine the function to correctly standardize the 'INFILTRATE' column from the beginning
def standardize_infiltrate(value):
    if pd.isnull(value):
        return 'missing'  # Representing missing data
    elif value in ['ΟΚ', 'OK']:
        return 'ok'       # Representing no significant infiltrates
    else:
        return value.strip().lower()  # Standardize the rest of the values

# Apply the redefined function to the 'INFILTRATE' column
dataset['INFILTRATE'] = dataset['INFILTRATE'].apply(standardize_infiltrate)

# Check the unique values after re-standardization
dataset['INFILTRATE'].unique()


array(['bilateral', 'r>l', 'l', 'ok', 'r', 'l>r', 'missing', 'fluid l'],
      dtype=object)

In [26]:
# Creating a dictionary for mapping the categories to numerical values
infiltrate_mapping = {
    'bilateral': 1,
    'r>l': 2,
    'l': 3,
    'ok': 4,
    'r': 5,
    'l>r': 6,
    'fluid l': 7
}

# Mapping the categories to numbers
dataset['INFILTRATE_numerical'] = dataset['INFILTRATE'].map(infiltrate_mapping)

# Display the first few rows to verify the transformation
dataset[['INFILTRATE', 'INFILTRATE_numerical']].head()

# Dropping the original 'INFILTRATE' column
dataset = dataset.drop('INFILTRATE', axis=1)

### Symptoms columns

In [27]:
# Define the translation dictionary
translation_dict = {
    'ΚΕΦΑΛΑΛΓΙΑ': 'HEADACHE',
    'ΑΝΟΣΜΙΑ': 'ANOSMIA',
    'ΝΑΥΤΙΑ': 'NAUSEA',
    'ΖΑΛΗ-ΑΣΤΑΘΕΙΑ-ΣΥΓΧΗΣΗ': 'DIZZINESS-INSTABILITY-CONFUSION',
    'ΠΛΕΥΡΙΤΙΚΗ ΣΥΛΛΟΓΗ': 'PLEURAL EFFUSION',
    'EMETOI': 'VOMITING'
}

# Define the symptom_columns list
symptom_columns = ['FEVER', 'COUGH', 'FATIGUE', 'DIARRHEAS', 'DYSPNEA', 'URTI']

# Step 1: Create columns for each symptom in symptom_columns
for symptom in symptom_columns:
    dataset[symptom + 'x'] = dataset[symptom].apply(lambda x: 1 if pd.notnull(x) and x else 0)

# Step 2: Translate the 'OTHER' column and create columns for each translated symptom
# Translate the 'OTHER' column
dataset['Translated_Other'] = dataset['OTHER'].map(translation_dict)

# Split the 'Translated_Other' into separate symptoms and create columns
for translated_symptom in translation_dict.values():
    dataset[translated_symptom] = dataset['Translated_Other'].apply(lambda x: 1 if x == translated_symptom else 0)

# Dropping the old columns
columns_to_drop = ['FEVER',	'COUGH',	'FATIGUE',	'DIARRHEAS',	'DYSPNEA',	'URTI',	'OTHER', 'Translated_Other']

dataset = dataset.drop(columns=columns_to_drop)


### Comorbilities

In [28]:
# Define the comorbidity columns list
comorbidity_columns_updated = [
    'CM/CHF', 'CM/CAD', 'CM/AF', 'CM/HBP', 'CM/asthma', 'CM/COPD', 'CM/DM',
    'DM REGISTRY', 'CM/neoplasm', 'CM/CNS', 'CM/GI', 'CM/renal', 
    'cm/autoimm', 'CM/lipid', 'CM/metabolic/other', 'CM/other'
]
# Create columns for each comorbidity with 1 for presence and 0 for absence
for comorbidity in comorbidity_columns_updated:
    if comorbidity != 'CM/other':
        dataset[comorbidity + '_Presence'] = dataset[comorbidity].apply(lambda x: 1 if pd.notnull(x) and x else 0)

# Display the first few rows to verify the transformation for these comorbidities
dataset[[comorbidity + '_Presence' for comorbidity in comorbidity_columns_updated if comorbidity != 'CM/other']].head()


,CM/CHF_Presence,CM/CAD_Presence,CM/AF_Presence,CM/HBP_Presence,CM/asthma_Presence,CM/COPD_Presence,CM/DM_Presence,DM REGISTRY_Presence,CM/neoplasm_Presence,CM/CNS_Presence,CM/GI_Presence,CM/renal_Presence,cm/autoimm_Presence,CM/lipid_Presence,CM/metabolic/other_Presence
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,1,1,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0


In [29]:
# Identifying unique values in the 'CM/other' column
unique_cm_other_values = dataset['CM/other'].dropna().unique()

# Filtering out combinatory values and creating a list of individual terms
individual_cm_other_values = [val for val in unique_cm_other_values if '/' not in val  and '-' not in val]

individual_cm_other_values

['ΚΥΠ',
 'Ψ',
 'ΧΛΛ',
 'ΗΒV',
 'ΧΘΕΝ ΑΝΕΥΡΥΣΜΑ ΚΟΙΛΙΑΚΗΣ ΑΟΡΤΗΣ',
 'ΛΕΜΦΩΜΑ',
 'ΑΝΑΙΜΙΑ',
 'ΒΑΛΒΙΔΟΠΛΑΣΤΙΚΗ ΑΟΡΤΙΚΗΣ',
 'ΑΚΜΗ',
 'HBV',
 'ΟΥΡΙΧΑΙΜΙΑ',
 'ΒΜΙ',
 'ΑΛΚΟΟΛ',
 'ΕΓΚΥΟΣ',
 'ΑΘΗΡΩΜΑΤΩΣΗ ΚΑΡΩΤΙΔΩΝ',
 'BMI',
 'ΛΕΧΩΝΑ',
 'PACEMAKER',
 'ΙΔΙΟΠΑΘΗΣ ΤΡΟΜΟΣ',
 'Β ΕΤΕΡΟΖΥΓΟΣ',
 'ΝΕΦΡΟΛΙΘΙΑΣΗ',
 'ΑΙΜΟΛΥΤΙΚΗ ΑΝΑΙΜΙΑ',
 'ΑΓΚΥΛΟΠΟΙΗΤΙΚΗ ΣΠΟΝΔΥΛΙΤΙΔΑ',
 'ΑΝΕΥΡΥΣΜΑ ΑΟΡΤΗΣ',
 'MDS',
 'ΠΟΛΛΑΠΛΟΥΝ ΜΥΕΛΩΜΑ',
 'ΣΑΡΚΟΕΙΔΩΣΗ',
 'ΠΕ',
 'ΜΕΤΑΛΛΙΚΗ ΒΑΛΒΙΔΑ',
 'ΕΤΕΡΟΖΥΓΟΣ V LEIDEN',
 'TB',
 'ΕΜΦΡΑΚΤΟ ΝΕΦΡΟΥ',
 'ΣΤΕΝΩΣΗ ΚΑΡΩΤΙΔΩΝ',
 'ΚΑΤΑΚΕΚΛΙΜΕΝΗ',
 'ΠΑΛΙΝΔΡΟΜΟΣ ΚΥΗΣΗ',
 'NON HODGKIN ',
 'ΝΟΗΤΙΚΗ ΥΣΤΕΡΗΣΗ']

In [30]:
# Complete translation dictionary for common medical abbreviations and terms in 'CM/other' column
# This includes the translations we identified earlier and leaves untranslated terms as-is
complete_medical_translation_dict = {
    'ΚΥΠ': 'Chronic Kidney Disease',
    'Ψ': 'Psychiatric Disorder',
    'ΧΛΛ': 'Chronic Lymphocytic Leukemia',
    'ΗΒV': 'Hepatitis B Virus',
    'ΧΘΕΝ ΑΝΕΥΡΥΣΜΑ ΚΟΙΛΙΑΚΗΣ ΑΟΡΤΗΣ': 'Abdominal Aortic Aneurysm',
    'ΛΕΜΦΩΜΑ': 'Lymphoma',
    'ΑΝΑΙΜΙΑ': 'Anemia',
    'ΒΑΛΒΙΔΟΠΛΑΣΤΙΚΗ ΑΟΡΤΙΚΗΣ': 'Aortic Valve Plasty',
    'ΑΚΜΗ': 'Acne',
    'HBV': 'Hepatitis B Virus',
    'ΟΥΡΙΧΑΙΜΙΑ': 'Uricemia',
    'ΒΜΙ': 'BMI',
    'ΑΛΚΟΟΛ': 'Alcohol',
    'ΕΓΚΥΟΣ': 'Pregnant',
    'ΑΘΗΡΩΜΑΤΩΣΗ ΚΑΡΩΤΙΔΩΝ': 'Carotid Atherosclerosis',
    'BMI': 'Body Mass Index',
    'ΛΕΧΩΝΑ': 'Leukemia',
    'PACEMAKER': 'Pacemaker',
    'ΙΔΙΟΠΑΘΗΣ ΤΡΟΜΟΣ': 'Idiopathic Tremor',
    'Β ΕΤΕΡΟΖΥΓΟΣ': 'Beta Thalassemia Heterozygous',
    'ΝΕΦΡΟΛΙΘΙΑΣΗ': 'Nephrolithiasis',
    'ΑΙΜΟΛΥΤΙΚΗ ΑΝΑΙΜΙΑ': 'Hemolytic Anemia',
    'ΑΓΚΥΛΟΠΟΙΗΤΙΚΗ ΣΠΟΝΔΥΛΙΤΙΔΑ': 'Ankylosing Spondylitis',
    'ΑΝΕΥΡΥΣΜΑ ΑΟΡΤΗΣ': 'Aortic Aneurysm',
    'MDS': 'Myelodysplastic Syndrome',
    'ΠΟΛΛΑΠΛΟΥΝ ΜΥΕΛΩΜΑ': 'Multiple Myeloma',
    'ΣΑΡΚΟΕΙΔΩΣΗ': 'Sarcoidosis',
    'ΠΕ': 'Pulmonary Embolism',
    'ΜΕΤΑΛΛΙΚΗ ΒΑΛΒΙΔΑ': 'Metallic Valve',
    'ΕΤΕΡΟΖΥΓΟΣ V LEIDEN': 'Heterozygous Factor V Leiden',
    'TB': 'Tuberculosis',
    'ΕΜΦΡΑΚΤΟ ΝΕΦΡΟΥ': 'Renal Infarction',
    'ΣΤΕΝΩΣΗ ΚΑΡΩΤΙΔΩΝ': 'Carotid Stenosis',
    'ΚΑΤΑΚΕΚΛΙΜΕΝΗ': 'Bedridden',
    'ΠΑΛΙΝΔΡΟΜΟΣ ΚΥΗΣΗ': 'Recurrent Pregnancy',
    'NON HODGKIN': 'Non-Hodgkin Lymphoma',
    'ΝΟΗΤΙΚΗ ΥΣΤΕΡΗΣΗ': 'Intellectual Disability'
}
# Apply the translations to the unique values in 'CM/other'
translated_cm_other_values = [complete_medical_translation_dict.get(val, val) for val in individual_cm_other_values]

# Creating columns for each translated value in 'CM/other'
for original_value, translated_value in zip(individual_cm_other_values, translated_cm_other_values):
    # Creating a column with the translated name
    column_name = translated_value.replace(" ", "_").replace("/", "_").replace("-", "_")
    dataset[column_name] = dataset['CM/other'].apply(lambda x: 1 if x == original_value else 0)
# Dropping the old columns
columns_to_drop = ['MED/cardio',	'MED/statins',	'MED/antiPLTs',	'MED/anticoag',	'MED/resp',	'MED/endocr',	'MED/DM',	'MED/CNS',	'MED/GI',	'MED/bones',	'MED/immuno',	'MED/antineo',	'MED/other',	'CM/CHF',	'CM/CAD',	'CM/AF',	'CM/HBP',	'CM/asthma',	'CM/COPD',	'CM/DM',	'DM REGISTRY',	'CM/neoplasm',	'CM/CNS',	'CM/hepatobiliary',	'CM/GI',	'CM/renal',	'cm/autoimm',	'CM/lipid',	'CM/metabolic/other',	'CM/other',]

dataset = dataset.drop(columns=columns_to_drop)


### Medications

In [31]:
# Re-defining the medication_columns list as it was lost during the re-upload of the file
medication_columns = [
    'ANTIBIOTIC 1', 'ANTIBIOTIC 2', 'ANTIBIOTIC 3', 'ANTIBIOTIC 4', 'ANTIBIOTIC 5', 
    'ANTIBIOTIC 6', 'ANTIBIOTIC 7', 'ANTIBIOTIC 8', 'ANTIBIOTIC 9', 'ANTICOAGULANT', 'ANTIFUNGAL 1', 'ANTIFUNGAL 2', 'OTHER2'
]

# Identifying unique values in medication columns
unique_medication_values = set()

for col in medication_columns:
    unique_medication_values.update(dataset[col].dropna().unique())

# Function to standardize medication names
def standardize_med_name(med_name, name_mapping):
    return name_mapping.get(med_name, med_name)

# Dictionary for mapping non-standard names to standard names
name_mapping = {
    'zirhromax': 'zithromax',
    'collisitn': 'colistin',
    'collisitin': 'colistin',
    'colisitn': 'colistin',
    'noradren/solumedrol': 'solumedrol',
    'nivestin/zovirax/solucortef': 'zovirax/solucortef',
    'vonvon': 'voncon',
    'DEXATON': 'dexamethasone',  # Assuming Dexaton is dexamethasone
    'collistin': 'colistin',
    'ΑΛΛΕΡΓΙΑ': 'allergia'  # Translating from Greek
    # Add more mappings as needed
}

# Standardizing unique medication values
standardized_unique_medication_values = set([standardize_med_name(med, name_mapping) for med in unique_medication_values])

# Displaying the standardized unique medication values
standardized_unique_medication_values

# Creating new columns for each standardized medication value
for med in standardized_unique_medication_values:
    if med != 1:  # Skipping columns that already indicate presence with a value of 1
        column_name = "Medication_" + str(med).replace(" ", "_").replace("/", "_").replace("-", "_")
        dataset[column_name] = dataset[medication_columns].apply(lambda row: 1 if med in row.values else 0, axis=1)

dataset = dataset.drop(columns=medication_columns)

# Displaying the first few rows to verify the new columns
dataset[[col for col in dataset.columns if col.startswith("Medication_")]].head()



,Medication_meronem,Medication_zebaxa,Medication_augmentin,Medication_minocin,Medication_zovirax_solucortef,Medication_ecalta,Medication_mefoxil,Medication_medrol,Medication_ambisome,Medication_rivaroxaban,...,Medication_zavicefta,Medication_vibramycin,Medication_begalin,Medication_fosfomycin,Medication_fondaparinux,Medication_colistin,Medication_tinzaparin,Medication_targocid,Medication_colchicine,Medication_tavanic
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Creating new columns for specified medications, replacing missing values with 0, and then dropping the old columns
specified_meds = ['REMDESIVIR', 'TAMIFLU', 'KALETRA', 'DEXATON', 'BARITICINIB', 'ANAKINRA', 
                  'TOCILIZUMAB', 'PLAQUENIL']

for med in specified_meds:
    new_column_name = med + "_Processed"
    dataset[new_column_name] = dataset[med].fillna(0)

# Dropping the old medication columns
dataset.drop(columns=specified_meds, inplace=True)


### Dropping columns 

In [33]:
# Dropping specified columns from the dataset
columns_to_drop = [ 'DAY OF ADMISSION', 'INTUBATION DATE', 'DATE CPAP', 'CPAP / INTUB', 'FA ΟΙΚΕΙ', 'ΑΝΤΙΒ ΑΓΩΓΗ ΝΟΣΟΚ', 'GSC2',	'BP2',	'BPM',	'rr2',	'TEMP6',	'PH7',	'PO28',	'PCO29',	'HCO310',	'FIO2 eisagwgh11',	'GSC22',	'BP23',	'BPM4',	'rr55',	'TEMP62',	'PH73',	'PO284',	'PCO295',	'HCO3106',	'FIO2 eisagwgh117', 'Στήλη1',	'Στήλη2',	'Στήλη3', 'Στήλη4', 'Στήλη5', 'Unnamed: 147',	'Unnamed: 148', 'CCI', 'PO2/FIO2', 'SOFA', 'APACHE', 'PSI']
dataset.drop(columns=columns_to_drop, inplace=True)

In [34]:
# Renaming the column "HIGH FLOW " to "HIGH FLOW"
dataset.rename(columns={"HIGH FLOW ": "HIGH FLOW"}, inplace=True)

In [35]:
# Checking for missing values in each row
missing_values_per_row = dataset.isnull().sum(axis=1)

# Creating a DataFrame to display the number of missing values in each row
row_missing_data_analysis = pd.DataFrame({
    'Row Index': missing_values_per_row.index, 
    'Missing Values in Row': missing_values_per_row.values
})

# Sorting the DataFrame based on the number of missing values in descending order
row_missing_data_analysis.sort_values(by='Missing Values in Row', ascending=False, inplace=True)

# Displaying the top 10 rows with the most missing values
row_missing_data_analysis.head(51)

,Row Index,Missing Values in Row
507,507,53
510,510,50
353,353,49
250,250,49
282,282,49
285,285,49
309,309,49
318,318,49
319,319,49
338,338,49


In [36]:
dataset = dataset.dropna(thresh=(dataset.shape[1]-30))

In [37]:
# List of specified columns
specified_columns = [
    "AGE", "GENDER", "LOS", "DAYS OF SYMPTOMS", "INTUBATION", "tINTUBATION", "CPAP", "tCPAP", 
    "HIGH FLOW", "t HIGH FLOW", "GSC", "BP", "PULSE RATE", "rr", "TEMP", "PH", "PO2", "PCO2", 
    "HCO3", "FIO2 eisagwgh", "WHO score", "WBC", "LDH", "CPK", "CRP", "FERRITIN", "PERNEUTROPHILS", 
    "PERLYMPHOCYTES", "ABSLYMPHOCYTES", "Hb", "PLT", "INR", "APTT", "FIBRINOGEN", "D-DIMERS", 
    "K", "NA", "Glu", "UREA", "CR", "BIL", "DIRECT BIL", "SGOT", "SGPT", "TnI", "CHOL", "TGL"
]

# Calculate the percentage of missing values for the specified columns
missing_values_percentage_specified = dataset[specified_columns].isnull().mean() * 100
missing_values_percentage_specified.sort_values(ascending=False).head(30)

t HIGH FLOW       98.301486
tINTUBATION       92.781316
tCPAP             88.535032
CHOL              84.925690
TGL               84.288747
rr                78.131635
HIGH FLOW         38.853503
FERRITIN          17.622081
FIBRINOGEN        17.197452
TnI               12.738854
D-DIMERS           9.978769
CPAP               9.341826
DIRECT BIL         7.006369
BIL                5.307856
INR                4.458599
APTT               4.458599
CRP                4.246285
PCO2               3.609342
HCO3               3.609342
WBC                3.609342
K                  3.397028
PO2                2.972399
ABSLYMPHOCYTES     2.760085
LDH                2.760085
PLT                2.547771
NA                 2.547771
Glu                2.335456
CR                 2.335456
SGOT               2.335456
SGPT               2.335456
dtype: float64

In [38]:
# Dropping the columns 'CHOL', 'TGL', and 'rr'
dataset = dataset.drop(columns=['CHOL', 'TGL', 'rr'])

### Missing Values into 0

In [39]:
# Adjusting the process to only replace missing values for columns that exist in the dataset
columns_to_replace_missing = ['tINTUBATION', 'CPAP', 'tCPAP', 'HIGH FLOW', 't HIGH FLOW']
existing_columns = dataset.columns

# Replacing missing values with 0 in the existing specified columns
for col in columns_to_replace_missing:
    if col in existing_columns:
        dataset[col] = dataset[col].fillna(0)



## Outlier handling

In [40]:
# Correcting the temperature values using Subject ID
dataset.loc[dataset['Subject_ID'] == 172, 'TEMP'] = 38.2
dataset.loc[dataset['Subject_ID'] == 313, 'TEMP'] = 36.1

# Correcting the PCO2 value for Subject ID 2
dataset.loc[dataset['Subject_ID'] == 2, 'PCO2'] = 31.2

# Setting the HCO3 value for Subject ID 305 as missing
dataset.loc[dataset['Subject_ID'] == 305, 'HCO3'] = pd.NA

# Setting the WHO score of 15 to missing value
dataset.loc[dataset['WHO score'] == 15, 'WHO score'] = pd.NA

# Setting the specified WBC values to missing for the given Subject IDs
subject_ids_to_update = [40, 41, 42, 43, 44, 45, 46, 47]
for subject_id in subject_ids_to_update:
    dataset.loc[dataset['Subject_ID'] == subject_id, 'WBC'] = pd.NA

# Setting the LDH value for Subject ID 479 to missing
dataset.loc[dataset['Subject_ID'] == 479, 'LDH'] = pd.NA

# Setting the CPK value for Subject ID 233 to missing
dataset.loc[dataset['Subject_ID'] == 233, 'CPK'] = pd.NA

# Setting the CRP value for Subject ID 364 to missing
dataset.loc[dataset['Subject_ID'] == 364, 'CRP'] = pd.NA

# Identifying the highest value of Hb
highest_hb_row = dataset['Hb'].idxmax()

# Setting this value to missing
dataset.loc[highest_hb_row, 'Hb'] = pd.NA

# Setting INR values over 10 to missing
dataset.loc[dataset['INR'] > 10, 'INR'] = pd.NA

# Setting APTT values over 200 to missing
dataset.loc[dataset['APTT'] > 200, 'APTT'] = pd.NA

# Identifying and setting the highest value of BIL to missing
highest_bil_row = dataset['BIL'].idxmax()
dataset.loc[highest_bil_row, 'BIL'] = pd.NA

# Setting Glu values over 600 to missing
dataset.loc[dataset['Glu'] > 600, 'Glu'] = pd.NA

# Setting UREA values over 500 to missing
dataset.loc[dataset['UREA'] > 500, 'UREA'] = pd.NA

# Identifying rows with DIRECT BIL values over 10
direct_bil_over_10 = dataset[dataset['DIRECT BIL'] > 10][['Subject_ID', 'DIRECT BIL']]

# Setting the identified high DIRECT BIL values to missing
for subject_id in direct_bil_over_10['Subject_ID']:
    dataset.loc[dataset['Subject_ID'] == subject_id, 'DIRECT BIL'] = pd.NA

# Identifying and setting the highest value of TnI (Troponin I) to missing
highest_tni_row = dataset['TnI'].idxmax()
dataset.loc[highest_tni_row, 'TnI'] = pd.NA

In [41]:
# Identifying outliers for FERRITIN
Q1_ferritin = dataset['FERRITIN'].quantile(0.25)
Q3_ferritin = dataset['FERRITIN'].quantile(0.75)
IQR_ferritin = Q3_ferritin - Q1_ferritin
lower_bound_ferritin = Q1_ferritin - 1.5 * IQR_ferritin
upper_bound_ferritin = Q3_ferritin + 1.5 * IQR_ferritin
outlier_rows_ferritin = dataset[(dataset['FERRITIN'] < lower_bound_ferritin) | (dataset['FERRITIN'] > upper_bound_ferritin)]

outlier_rows_ferritin[['Subject_ID', 'FERRITIN']]

# Identifying the three highest values of FERRITIN
three_highest_ferritin = outlier_rows_ferritin.nlargest(3, 'FERRITIN')

# Setting these values to missing
for subject_id in three_highest_ferritin['Subject_ID']:
    dataset.loc[dataset['Subject_ID'] == subject_id, 'FERRITIN'] = pd.NA

In [42]:
# Identifying and setting the highest value of FIBRINOGEN to missing
dataset['FIBRINOGEN'] = pd.to_numeric(dataset['FIBRINOGEN'], errors='coerce')
highest_fibrinogen_row = dataset['FIBRINOGEN'].idxmax(skipna=True)
dataset.loc[highest_fibrinogen_row, 'FIBRINOGEN'] = pd.NA

In [43]:
# Identifying outliers for K with values above 100
outlier_rows_k_above_100 = dataset[dataset['K'] > 100][['Subject_ID', 'K']]

# Identifying outliers for NA with values above 1000
outlier_rows_na_above_1000 = dataset[dataset['NA'] > 1000][['Subject_ID', 'NA']]


# Setting the identified outliers for K and NA to missing values
subject_ids_k_outliers = outlier_rows_k_above_100['Subject_ID']
subject_ids_na_outliers = outlier_rows_na_above_1000['Subject_ID']

for subject_id in subject_ids_k_outliers:
    dataset.loc[dataset['Subject_ID'] == subject_id, 'K'] = pd.NA

for subject_id in subject_ids_na_outliers:
    dataset.loc[dataset['Subject_ID'] == subject_id, 'NA'] = pd.NA

In [44]:
# Cleaning the "D-DIMERS" column by converting non-standard entries like '15+' and '15+++' to numeric values (e.g., '15+'' to 15)
# We will use a regular expression to extract the numeric part from these strings

import re

D_DIMERS_column_index = dataset.columns.get_loc('D-DIMERS')

def clean_d_dimers(value):
    if isinstance(value, str):
        # Extract the numeric part from the string
        match = re.search(r'\d+', value)
        return float(match.group()) if match else None
    else:
        return value

dataset['D-DIMERS_cleaned'] = dataset['D-DIMERS'].apply(clean_d_dimers)

# Converting the cleaned column to numeric type
dataset['D-DIMERS_cleaned'] = pd.to_numeric(dataset['D-DIMERS_cleaned'], errors='coerce')

# Checking for missing values after cleaning and conversion
cleaned_missing_values = dataset['D-DIMERS_cleaned'].isnull().sum()

# Dropping the original 'D-DIMERS' column
dataset.drop('D-DIMERS', axis=1, inplace=True)

# Displaying basic descriptive statistics of the cleaned "D-DIMERS" column
cleaned_d_dimers_stats = dataset['D-DIMERS_cleaned'].describe()

dataset.insert(D_DIMERS_column_index, 'D-DIMERS_cleaned', dataset.pop('D-DIMERS_cleaned'))

# Identifying and setting the highest value of D-DIMERS_cleaned to missing
highest_ddimers_row = dataset['D-DIMERS_cleaned'].idxmax(skipna=True)
dataset.loc[highest_ddimers_row, 'D-DIMERS_cleaned'] = pd.NA


In [45]:
# Cleaning the "FIO2 eisagwgh" column by keeping only the numeric part from the entries
FIO2_eisagwgh_column_index = dataset.columns.get_loc('FIO2 eisagwgh')
def clean_fio2(value):
    if isinstance(value, str):
        # Extract the numeric part from the string
        match = re.search(r'\d+', value)
        return float(match.group()) if match else None
    else:
        return value

dataset['FIO2 eisagwgh_cleaned'] = dataset['FIO2 eisagwgh'].apply(clean_fio2)

# Converting the cleaned column to numeric type
dataset['FIO2 eisagwgh_cleaned'] = pd.to_numeric(dataset['FIO2 eisagwgh_cleaned'], errors='coerce')

# Checking for missing values after cleaning and conversion
cleaned_fio2_missing_values = dataset['FIO2 eisagwgh_cleaned'].isnull().sum()

# Dropping the original 'FIO2 eisagwgh' column
dataset.drop('FIO2 eisagwgh', axis=1, inplace=True)

# Displaying basic descriptive statistics of the cleaned "FIO2 eisagwgh" column
cleaned_fio2_stats = dataset['FIO2 eisagwgh_cleaned'].describe()

dataset.insert(FIO2_eisagwgh_column_index, 'FIO2 eisagwgh_cleaned', dataset.pop('FIO2 eisagwgh_cleaned'))


In [46]:
GSC_column_index = dataset.columns.get_loc('GSC')
# Treating entries with 'Δ' in the "GSC" column as missing values
dataset['GSC_cleaned'] = dataset['GSC'].replace('Δ', None)

# Converting the cleaned "GSC" column to a numeric type
dataset['GSC_cleaned'] = pd.to_numeric(dataset['GSC_cleaned'], errors='coerce')

# Checking for missing values after cleaning and conversion
cleaned_gsc_missing_values = dataset['GSC_cleaned'].isnull().sum()

# Dropping the original 'GSC' column
dataset.drop('GSC', axis=1, inplace=True)

# Displaying basic descriptive statistics of the cleaned "GSC" column
cleaned_gsc_stats = dataset['GSC_cleaned'].describe()

dataset.insert(GSC_column_index, 'GSC_cleaned', dataset.pop('GSC_cleaned'))


In [47]:
# Finding the position of the original "BP" column
bp_column_index = dataset.columns.get_loc('BP')

# Splitting the "BP" column into two separate columns for systolic and diastolic values
bp_split = dataset['BP'].str.split('/', expand=True)
dataset['Systolic_BP'] = pd.to_numeric(bp_split[0], errors='coerce')
dataset['Diastolic_BP'] = pd.to_numeric(bp_split[1], errors='coerce')

# Dropping the original "BP" column
dataset.drop('BP', axis=1, inplace=True)

# Inserting the new systolic and diastolic BP columns in the position where the original BP column was
dataset.insert(bp_column_index, 'Diastolic_BP', dataset.pop('Diastolic_BP'))
dataset.insert(bp_column_index, 'Systolic_BP', dataset.pop('Systolic_BP'))

# Replacing the unusually high value in 'Systolic_BP' with NaN
dataset.loc[dataset['Subject_ID'] == 367, 'Systolic_BP'] = pd.NA


In [48]:
tINTUBATION_column_index = dataset.columns.get_loc('tINTUBATION')

# Applying the cleaning process for "tINTUBATION"
def clean_tintubation(value):
    if isinstance(value, str):
        # Extract the numeric part from the string
        match = re.search(r'\d+', value)
        return float(match.group()) if match else None
    else:
        return value

dataset['tINTUBATION_cleaned'] = dataset['tINTUBATION'].apply(clean_tintubation)

# Converting the cleaned column to numeric type
dataset['tINTUBATION_cleaned'] = pd.to_numeric(dataset['tINTUBATION_cleaned'], errors='coerce')

# Checking for missing values after cleaning and conversion
cleaned_tintubation_missing_values = dataset['tINTUBATION_cleaned'].isnull().sum()

# Dropping the original tINTUBATION column
dataset.drop('tINTUBATION', axis=1, inplace=True)

dataset.insert(tINTUBATION_column_index, 'tINTUBATION_cleaned', dataset.pop('tINTUBATION_cleaned'))

In [49]:
# Replace "ΑΛΛΕΡΓΙΑ" with 0 in the original dataframe
dataset["REMDESIVIR_Processed"] = dataset["REMDESIVIR_Processed"].replace("ΑΛΛΕΡΓΙΑ", 0)

# Now convert the "REMDESIVIR_Processed" column to numeric, coercing any errors into NaN
dataset["REMDESIVIR_Processed"] = pd.to_numeric(dataset["REMDESIVIR_Processed"], errors='coerce')

# Ensure there are no NaN values left in the "REMDESIVIR_Processed" column
remaining_nan_values = dataset["REMDESIVIR_Processed"].isnull().sum()
remaining_nan_values, dataset["REMDESIVIR_Processed"].unique()


(0, array([0, 1]))

In [50]:
# Replacing implausible pH values with NaN
dataset.loc[dataset['PH'] == 36.10, 'PH'] = pd.NA
dataset.loc[dataset['PH'] == 73.41, 'PH'] = pd.NA
# Converting the values in 'PH' column that are below 7 to missing values (NaN)
dataset.loc[dataset['PH'] < 7, 'PH'] = None


# Missing value Imputation

In [52]:
# Calculate the percentage of missing values per column
missing_values_percentage = dataset.isnull().mean() * 100

# Display the results
missing_values_percentage_sorted = missing_values_percentage.sort_values(ascending=False)
missing_values_percentage_sorted.head(30)

Στήλη6              58.811040
qSOFA               21.443737
FERRITIN            18.259023
FIBRINOGEN          17.409766
TnI                 12.951168
D-DIMERS_cleaned    10.191083
DIRECT BIL           9.766454
APTT                 6.369427
INR                  6.157113
BIL                  5.520170
K                    5.520170
WBC                  5.307856
CRP                  4.458599
PH                   4.033970
HCO3                 3.821656
PCO2                 3.609342
PO2                  2.972399
NA                   2.972399
LDH                  2.972399
Glu                  2.760085
ABSLYMPHOCYTES       2.760085
UREA                 2.547771
PLT                  2.547771
Hb                   2.547771
PERNEUTROPHILS       2.335456
PERLYMPHOCYTES       2.335456
SGOT                 2.335456
CR                   2.335456
SGPT                 2.335456
CPK                  2.335456
dtype: float64

In [ ]:
index_columns = ['Subject_ID',	'SAMPLE No',	'Στήλη6', 'ΑΜ ΡΙΟΥ']
dataset1_columns = [col for col in dataset if col not in index_columns]
dataset1 = dataset[dataset1_columns]
# Columns of interest for imputation
columns_of_interest = [
    "AGE", "GENDER", "LOS", "DAYS OF SYMPTOMS", "INTUBATION", "tINTUBATION_cleaned", "CPAP",
    "tCPAP", "HIGH FLOW", "t HIGH FLOW", "GSC_cleaned", "Systolic_BP", "Diastolic_BP", "PULSE RATE", 
    "TEMP", "PH", "PO2", "PCO2", "HCO3", "FIO2 eisagwgh_cleaned", "WHO score", "WBC", "LDH", 
    "CPK", "CRP", "FERRITIN", "PERNEUTROPHILS", "PERLYMPHOCYTES", "ABSLYMPHOCYTES", "Hb", 
    "PLT", "INR", "APTT", "FIBRINOGEN", "D-DIMERS_cleaned", "K", "NA", "Glu", "UREA", "CR", 
    "BIL", "DIRECT BIL", "SGOT", "SGPT", "TnI", "qSOFA", "Outcome_numerical"
]

'\n# Extracting the columns of interest from the dataset\ndata_of_interest = dataset[columns_of_interest]\n\n# Checking for missing values\nmissing_values = data_of_interest.isnull().sum()\nmissing_values[missing_values > 0]  # Display only columns with missing values\n#for col in columns_of_interest:\n#    data_of_interest[col] = pd.to_numeric(data_of_interest[col])\nfor col in columns_of_interest:\n    data_of_interest.loc[:, col] = pd.to_numeric(data_of_interest[col])\n'

In [41]:
# Extracting the columns of interest from the dataset and creating a copy to avoid SettingWithCopyWarning
data_of_interest = dataset1[columns_of_interest].copy()

# Checking for missing values
missing_values = data_of_interest.isnull().sum()
missing_values[missing_values > 0]  # Display only columns with missing values

# Converting columns to numeric, handling errors by converting them to NaN
for col in columns_of_interest:
    data_of_interest.loc[:, col] = pd.to_numeric(data_of_interest[col], errors='coerce')

In [42]:
# Define discrete and continuous columns 
discrete_columns = ['GENDER', 'INTUBATION', 'CPAP', 'HIGH FLOW', 'WHO score', "qSOFA", "Outcome_numerical"]
discrete_data = data_of_interest[discrete_columns]
continuous_columns = [col for col in columns_of_interest if col not in discrete_columns]
continuous_data = data_of_interest[continuous_columns] 

### Random forest option

In [ ]:
from sklearn.experimental import enable_iterative_imputer

# Create the IterativeImputer with RandomForestRegressor as the estimator
imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=5, max_depth= None, random_state=42),
                            max_iter=100, random_state=42)

# Perform the imputation
data_imputed = imputer.fit_transform(continuous_data)

# Convert the imputed data back to a DataFrame
data_imputed_df = pd.DataFrame(data_imputed, columns=continuous_columns)

data_imputed_df.index = dataset.index

data_imputed_df.index = dataset1.index

# Replace the original continuous columns in the dataset with the imputed data
for column in continuous_columns:
    dataset[column] = data_imputed_df[column]  
    dataset1.loc[:, column] = data_imputed_df[column]  



In [45]:
# Function to impute missing values using RandomForestClassifier for discrete data
def impute_numerical_discrete_rf(data, column, other_columns):
    # Prepare the training data (where column is not missing)
    train = data[data[column].notnull()]
    test = data[data[column].isnull()]

    if not test.empty:
        X_train = train[other_columns]
        y_train = train[column].astype('int')  # Ensure the target is integer
        X_test = test[other_columns]

        # Initialize and train classifier
        clf = RandomForestClassifier(n_estimators=100, random_state=42)
        clf.fit(X_train, y_train)

        # Predict and fill missing values
        predicted_values = clf.predict(X_test)
        data.loc[data[column].isnull(), column] = predicted_values

    return data

# List of other columns to use as predictors; typically all other columns except the one being imputed
other_columns = [col for col in dataset1.columns if col not in discrete_columns]

# Apply the imputation for each discrete column
for col in discrete_columns:
    dataset1 = impute_numerical_discrete_rf(dataset1, col, other_columns)
    dataset[col] = dataset1[col]

### KNN option

In [ ]:
'''
# Applying KNN imputation
knn_imputer = KNNImputer(n_neighbors=5)
data_imputed = knn_imputer.fit_transform(data_of_interest)

# Creating a DataFrame from the imputed data
data_imputed_df = pd.DataFrame(data_imputed, columns=columns_of_interest)

# Checking if any missing values remain
missing_values_after_imputation = data_imputed_df.isnull().sum()
missing_values_after_imputation[missing_values_after_imputation > 0]  # Display only columns with remaining missing values
'''

In [ ]:
'''
data_imputed_df.index = dataset.index
# Columns to be replaced with imputed data
columns_to_replace = data_imputed_df.columns
# Replace the original columns in the dataset with the imputed columns
for column in columns_to_replace:
    dataset[column] = data_imputed_df[column]
'''

In [ ]:
# Exporting progress 
file_path_dataset13 = '/Complete_data.xlsx' 
dataset.to_excel(file_path_dataset13, index=False)

# Merging the Dataset with CCI data

In [50]:
file2_path = '/Users/michael/Thesis project/Dataset/arxeio CCI.xlsx'
dataset_CCI = pd.read_excel(file2_path)

# Keeping rows where 'SAMPLE No' is not missing for each duplicate group in 'ΑΜ ΡΙΟΥ'
# First, sorting by 'SAMPLE No' so that NaNs go to the end, then dropping duplicates while keeping the first
data_sorted = dataset_CCI.sort_values(by=['ΑΜ ΡΙΟΥ', 'SAMPLE No'], na_position='last')
dataset_CCI = data_sorted.drop_duplicates(subset='ΑΜ ΡΙΟΥ', keep='first')

dataset_CCI = dataset_CCI.drop(columns=['SAMPLE No',	'ΠΕΙΡΑΜΑ',	'FACs'])

final_merge_missing_values = dataset_CCI.isnull().sum()
final_merge_duplicates = dataset_CCI.duplicated().sum()
final_merge_missing_values, final_merge_duplicates

(ΑΜ ΡΙΟΥ    1
 CCI        0
 dtype: int64,
 0)

In [51]:
# Merging the two tables
dataset = pd.merge(dataset, dataset_CCI, on='ΑΜ ΡΙΟΥ', how='left')

# Locate the row with 'Subject_ID' 497 and update the 'CCI' value to 3, this is the only replicated value
dataset.loc[dataset['Subject_ID'] == 497, 'CCI'] = 3
dataset.loc[dataset['Subject_ID'] == 370, 'Στήλη6'] = 'FACs'


In [ ]:
# Exporting progress 
file_path_dataset = '/preprocessed_COVID19_dataset_single.xlsx' 
dataset.to_excel(file_path_dataset, index=False)

# Merging with Cytokines & Cytometry Data

In [ ]:
# Adjusting the filter to include 'FACs'
filtered_data_corrected = dataset[dataset['Στήλη6'].isin([1, 'FACs'])]

# Saving the correctly filtered data to a new sheet in the Excel file
output_file_path_corrected = '/filtered_COVID19_dataset_with_Cyto_corrected.xlsx'
with pd.ExcelWriter(output_file_path_corrected, engine='xlsxwriter') as writer:
    dataset.to_excel(writer, sheet_name='Original Data', index=False)
    filtered_data_corrected.to_excel(writer, sheet_name='Cyto', index=False)

In [ ]:
# Load the additional datasets
file_path_cytometry = '/all_data_cytometry_patients.xlsx'
file_path_cytokines = '/Cytokines_all_patients.xlsx'
file_path_cyto = '/filtered_COVID19_dataset_with_Cyto_corrected.xlsx'
cyto_data = pd.read_excel(file_path_cyto, sheet_name='Cyto')
cytometry_data = pd.read_excel(file_path_cytometry)
# Drop 'Column1'
cytometry_data.drop(columns='Column1', inplace=True)
cytokines_data = pd.read_excel(file_path_cytokines)

# Merging the datasets
merged1 = pd.merge(cytokines_data,  cytometry_data, on='SAMPLE No', how='left')
merged_Cyto = pd.merge(cyto_data, merged1, on='SAMPLE No', suffixes=('_cytometry', '_cytokines'), how='left')


In [55]:
dataset.drop(columns=['Στήλη6', 'SAMPLE No',	'ΑΜ ΡΙΟΥ'], inplace=True)
merged_Cyto.drop(columns=['Στήλη6', 'SAMPLE No',	'ΑΜ ΡΙΟΥ'], inplace=True)

## Exporting Dataframe

In [ ]:
# Exporting the DataFrame to a excel file
exported_file_path = '/final_preprocessed_COVID19_dataset.xlsx' 
with pd.ExcelWriter(exported_file_path, engine='xlsxwriter') as writer:
    dataset.to_excel(writer, sheet_name='Original Data', index=False)
    merged_Cyto.to_excel(writer, sheet_name='Cyto', index=False)
